In [ ]:
!pip install -q pyrsgis rasterio

In [ ]:
# Bibliotecas
import os
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import rasterio as rio
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [ ]:
# Arquivos
area_treino_teste = '/content/drive/MyDrive/Sentinel-2/pnb_queimada.tif'
poligono_queimada = '/content/drive/MyDrive/Sentinel-2/queimada.tif'
area_total = '/content/drive/MyDrive/Sentinel-2/DF_S2_mosaico.tif'

# Lendo como array
raster1, treino = raster.read(area_treino_teste, bands='all')
raster2, queimada = raster.read(poligono_queimada, bands=1)
raster3, total = raster.read(area_total, bands='all')

# Substituindo NoData por zero nos dados rotulados
queimada = (queimada == 1).astype(int)

In [ ]:
# Redimensionando dados para 1-D
treino = changeDimension(treino)
total = changeDimension(total)
queimada = changeDimension(queimada)

# Split treino e test
xTrain, xTest, yTrain, yTest = train_test_split(treino, 
                                                queimada, test_size=0.4)
# Redimensionando os dados para o shape que o keras trabalha
xTrain = xTrain.reshape((xTrain.shape[0], 1, xTrain.shape[1]))
xTest = xTest.reshape((xTest.shape[0], 1, xTest.shape[1]))
total = total.reshape((total.shape[0], 1, total.shape[1]))

The "changeDimension()" function has moved to "array_to_table()" and will be deprecated in future versions. Please check the pyrsgis documentation at https://pyrsgis.readthedocs.io/en/master/ for more details.
The "changeDimension()" function has moved to "array_to_table()" and will be deprecated in future versions. Please check the pyrsgis documentation at https://pyrsgis.readthedocs.io/en/master/ for more details.
The "changeDimension()" function has moved to "array_to_table()" and will be deprecated in future versions. Please check the pyrsgis documentation at https://pyrsgis.readthedocs.io/en/master/ for more details.


In [ ]:
# Parametrizando a rede
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, treino.shape[1])),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16),
    keras.layers.Dense(8),
    keras.layers.Dense(4),
    keras.layers.Dense(2, activation='softmax')])

# Compilando modelo
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Rodando modelo
model.fit(xTrain, yTrain, epochs=5)

Epoch 1/5
127032/127032 [==============================] - 236s 2ms/step - loss: 0.0337 - accuracy: 0.9914
Epoch 2/5
127032/127032 [==============================] - 226s 2ms/step - loss: 0.0127 - accuracy: 0.9951
Epoch 3/5
127032/127032 [==============================] - 212s 2ms/step - loss: 0.0116 - accuracy: 0.9956
Epoch 4/5
127032/127032 [==============================] - 225s 2ms/step - loss: 0.0110 - accuracy: 0.9959
Epoch 5/5
127032/127032 [==============================] - 231s 2ms/step - loss: 0.0105 - accuracy: 0.9962


In [ ]:
# Realizando predição em dados de teste
predict_teste = model.predict(xTest)
predict_teste = predict_teste[:,1].astype(int)

# Calculando métricas de avaliação
precision = precision_score(yTest, predict_teste)
recall = recall_score(yTest, predict_teste)

84688/84688 [==============================] - 102s 1ms/step


In [ ]:
# Realizando predição na imagem geral
predict_total = model.predict(total)
predict_total = predict_total[:,1]

# Gravando resultado em disco
predict_final = np.reshape(predict_total,(raster3.RasterYSize, raster3.RasterXSize))
out_File = 'queimada_predict.tif'
raster.export(predict_final, raster3, filename=out_File, dtype='float')

2171272/3767513 [================>.............] - ETA: 35:35